## Методы сбора и обработки данных из сети Интернет
### Урок 4. Парсинг HTML. XPath

    1. Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
        ◦ название источника;
        ◦ наименование новости;
        ◦ ссылку на новость;
        ◦ дата публикации.

    2. Сложить собранные новости в БД

In [12]:
#  загружаемые библиотеки
from pprint import pprint
import requests
from lxml import html
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

In [13]:
#  Подключение базы данных MongoDB
client = MongoClient('localhost', 27017)

db = client['news']  # database
lenta = db.lenta  # collection

In [14]:
# #  Очистка коллекции БД:
# lenta.delete_many({})

In [15]:
url = 'https://lenta.ru/'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 '
                         'Safari/537.36'}

res = requests.get(url, headers=headers)
dom = html.fromstring(res.text)

topnews = dom.xpath("//a[contains(@class, '_topnews')]")

In [16]:
for item in topnews:
    source = url
    title = item.xpath('.//h3[contains(@class, "card-big__title")]//text() | .//span[contains(@class, '
                       '"card-mini__title")]//text()')
    link = item.xpath('./@href')[0]
    full_link = url + link
    data = item.xpath('.//time[contains(@class, "card-big__date")]//text() | .//time[contains(@class, '
                      '"card-mini__date")]//text()')
    news_list = {
        'Источник': url,
        'Название': title,
        'Ссылка': full_link,
        'Дата': data,
    }

    #  Запись документов в БД:
    try:
        lenta.update_one(news_list, {'$set': news_list}, upsert=True)
    except dke:
        print('Duplicate key error collection')

In [17]:
#  Просмотр результата:
for doc in lenta.find({}):
    pprint(doc)
print('Documents in collection: ', db.lenta.count_documents({}))  # Выводим количество документов в БД

{'_id': ObjectId('621d853589dba884ec37e5ed'),
 'Дата': ['02:51'],
 'Источник': 'https://lenta.ru/',
 'Название': ['Байден оценил вероятность ядерной войны'],
 'Ссылка': 'https://lenta.ru//news/2022/03/01/yadernaya/'}
{'_id': ObjectId('621d853589dba884ec37e5ef'),
 'Дата': ['05:26'],
 'Источник': 'https://lenta.ru/',
 'Название': ['Названы советы по восстановлению обоняния после COVID-19'],
 'Ссылка': 'https://lenta.ru/https://moslenta.ru/news/lyudi/vosstanovlenie-obonyaniya-01-03-2022.htm/'}
{'_id': ObjectId('621d853589dba884ec37e5f1'),
 'Дата': ['05:25'],
 'Источник': 'https://lenta.ru/',
 'Название': ['Harley-Davidson приостановила поставки мотоциклов в Россию'],
 'Ссылка': 'https://lenta.ru//news/2022/03/01/herley/'}
{'_id': ObjectId('621d853589dba884ec37e5f3'),
 'Дата': ['05:18'],
 'Источник': 'https://lenta.ru/',
 'Название': ['На Украине отключили связь для телефонов с рядом номеров'],
 'Ссылка': 'https://lenta.ru//news/2022/03/01/phones/'}
{'_id': ObjectId('621d853589dba884ec37e5